In [9]:
from numpy import genfromtxt
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets as ds
import sklearn.model_selection as ms
import sklearn.svm as svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

In [71]:
gesture = ['Down', 'Up', 'Left', 'Right']
path = 'HeadData/'
gyro = '/gyro/'
features = 4
trials = 40
x = [[0] * features for i in range(trials*len(gesture))]
y = [0] * trials*len(gesture) 

In [72]:
def min_max_feature_order(s1,s2):
    if (s1>s2): 
        return 1
    else:
        return 5

In [73]:
for i in range(0,len(gesture),1):
    for k in range(0,trials,1):
        path_data = path+ gesture[i]+ gyro +'trial'+str(k+1)+'.csv'
        data = genfromtxt(path_data, delimiter=',')
        #Features
        x[k+i*trials][0]=min_max_feature_order(np.argmax(data[:,1]), np.argmin(data[:,1]))
        x[k+i*trials][1]=min_max_feature_order(np.argmax(data[:,3]), np.argmin(data[:,3]))
        x[k+i*trials][2]=min(data[:,1]) # down up gestures the x dimension changes the most
        x[k+i*trials][3]=min(data[:,3]) # right left gestures the z dimension changes the most
       # x[k+i*trials][4]=max(data[:,1]) # down up gestures the x dimension changes the most
       # x[k+i*trials][5]=max(data[:,3]) # right left gestures the z dimension changes the most
        # x[k+i*trials][6]=data[:,1].mean() # mean for x dimension
        # x[k+i*trials][7]=data[:,3].mean() # mean for z dimension
        # x[k+i*trials][8]=data[:,1].std() # std for x dimension
        # x[k+i*trials][9]=data[:,3].std() # std for z dimension
        #target variables
        y[[k+i*trials][0]]=i

In [74]:
# Split dataset into training set and test set 
# 75% training and 25% test
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.25, random_state=0) 

In [75]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)        

In [76]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred,  average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred,  average='micro'))

# Model Confusion Matrix:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
Confusion Matrix:
[[ 8  0  0  0]
 [ 0 11  0  0]
 [ 0  0 11  0]
 [ 0  0  0 10]]


In [77]:
accuracies = cross_val_score(estimator = clf, X = X_train, y = y_train, cv = 4)
for i in range (0,4,1):
    print (accuracies[i])

print ('mean k-Fold cross-validation : ' , accuracies.mean())

1.0
1.0
1.0
1.0
mean k-Fold cross-validation :  1.0
